In [1]:
import os, csv
import time
import warnings
import numpy as np
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, pooling, BatchNormalization
from keras.utils import np_utils
from keras import backend as K
from matplotlib import pyplot as plt
from IPython.display import clear_output

Using TensorFlow backend.


In [2]:
num_cores = 24

num_CPU = 24
num_GPU = 4

config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
session = tf.Session(config=config)
K.set_session(session)

from keras.layers import merge
from keras.layers.core import Lambda
from keras.models import Model

import tensorflow as tf

def make_parallel(model, gpu_count):
    def get_slice(data, idx, parts):
        shape = tf.shape(data)
        size = tf.concat([ shape[:1] // parts, shape[1:] ],axis=0)
        stride = tf.concat([ shape[:1] // parts, shape[1:]*0 ],axis=0)
        start = stride * idx
        return tf.slice(data, start, size)

    outputs_all = []
    for i in range(len(model.outputs)):
        outputs_all.append([])

    #Place a copy of the model on each GPU, each getting a slice of the batch
    for i in range(gpu_count):
        with tf.device('/gpu:%d' % i):
            with tf.name_scope('tower_%d' % i) as scope:

                inputs = []
                #Slice each input into a piece for processing on this GPU
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_n = Lambda(get_slice, output_shape=input_shape, arguments={'idx':i,'parts':gpu_count})(x)
                    inputs.append(slice_n)                

                outputs = model(inputs)
                
                if not isinstance(outputs, list):
                    outputs = [outputs]
                
                #Save all the outputs for merging back together later
                for l in range(len(outputs)):
                    outputs_all[l].append(outputs[l])

    # merge outputs on CPU
    with tf.device('/cpu:0'):
        merged = []
        for outputs in outputs_all:
            merged.append(merge(outputs, mode='concat', concat_axis=0))
            
        return Model(input=model.inputs, output=merged)

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy Numpy warnings

In [4]:
def extract_col(filename,colnum): #colnum starts from 1 
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        a = list(reader)
        data = np.array([[[float(a[i][j])for j in range(colnum,colnum+1)]]for i in range(len(a))],float) # acc & gyr
        return data 

In [5]:
def load_data(filename, colnum, seq_len, normalise_window):
    data = extract_col(filename, colnum)

    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])

    if normalise_window:
#        result = normalise_windows(result)
        result = normalise_windows2(result)

    result = np.array(result)

    row = round(0.8 * result.shape[0]) #train first 90%, test on the rest 10% 
    train = result[:int(row), :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    return [x_train, y_train, x_test, y_test]

In [6]:
def normalise_windows(window_data):
    normalised_data = []
    for window in window_data:
        if ( float(window[0])== 0 ) :
            normalised_window = [((float(p) / 0.00001 ) - 1) for p in window] # float(window[0]) ==0 이란 소리로군
        else :
            normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

def normalise_windows2(window_data):
    normalised_data = []
    for window in window_data:
        normalised_window = [ (float(p)) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

In [7]:
def build_model(layers):
    model = Sequential()

    model.add(LSTM(
        input_shape=(layers[1], layers[0]),
        output_dim=layers[1],
        return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
        layers[2],
        return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(
        output_dim=layers[3]))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop")
    print("> Compilation Time : ", time.time() - start)
    return model

In [8]:
def predict_point_by_point(model, data):
    #Predict each timestep given the last sequence of true data, in effect only predicting 1 step ahead each time
    predicted = model.predict(data)
    predicted = np.reshape(predicted, (predicted.size,))
    return predicted

In [9]:
def predict_sequence_full(model, data, window_size):
    #Shift the window by 1 new prediction each time, re-run predictions on new window
    curr_frame = data[0]
    predicted = []
    for i in range(len(data)):
        predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
        curr_frame = curr_frame[1:]
        curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
    return predicted

In [10]:
def predict_sequences_multiple(model, data, window_size, prediction_len):
    #Predict sequence of 50 steps before shifting prediction run forward by 50 steps
    prediction_seqs = []
    for i in range(int(len(data)/prediction_len)):
        curr_frame = data[i*prediction_len]
        predicted = []
        for j in range(prediction_len):
            predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
            curr_frame = curr_frame[1:]
            curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
    return prediction_seqs

In [11]:
def plot_results(predicted_data, true_data):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    plt.plot(predicted_data, label='Prediction')
    plt.legend()
    plt.show()


In [12]:
def plot_results_multiple(predicted_data, true_data, prediction_len):
    fig = plt.figure(facecolor='white',figsize=(100,30))

    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    #Pad the list of predictions to shift it in the graph to it's correct start
    for i, data in enumerate(predicted_data):
        padding = [None for p in range(i * prediction_len)]
        plt.plot(padding + data, label='Prediction')
        plt.legend()
    plt.show()

In [13]:
#Main Run Thread
if __name__=='__main__':
	global_start_time = time.time()
	epochs  = 1
	seq_len = 250

	print('> Loading data... ')

	X_train, y_train, X_test, y_test = load_data('fall_yes.csv',2, seq_len, True)

	print('> Data Loaded. Compiling...')

	model = build_model([1, 250, 100, 1])

	model.fit(
	    X_train,
	    y_train,
	    batch_size=512,
	    nb_epoch=epochs,
	    validation_split=0.05)

	predictions = predict_sequences_multiple(model, X_test, seq_len, 250)
#	predicted = lstm.predict_sequence_full(model, X_test, seq_len)
#	predicted = lstm.predict_point_by_point(model, X_test)

	print('Training duration (s) : ', time.time() - global_start_time)
    
	plot_results_multiple(predictions, y_test, 250)
print(predictions)


> Loading data... 
> Data Loaded. Compiling...
> Compilation Time :  0.01922607421875
Train on 604410 samples, validate on 31812 samples
Epoch 1/1
604410/604410 [==============================] - 6084s 10ms/step - loss: 308.9698 - val_loss: 81.6688
Training duration (s) :  15520.407860517502


[[58.064423, 56.966732, 54.942703, 52.280655, 49.418438, 46.658997, 44.057777, 41.455334, 38.86168, 36.188858, 33.304443, 30.146471, 26.786825, 23.364614, 19.996283, 16.87951, 14.1345625, 11.843546, 10.015659, 8.689109, 7.9304557, 7.7484894, 8.117649, 8.949422, 10.074415, 11.309346, 12.495557, 13.509777, 14.2725315, 14.742134, 14.922291, 14.845612, 14.564293, 14.137847, 13.622254, 13.065094, 12.504557, 11.971756, 11.485674, 11.059208, 10.698571, 10.404535, 10.173246, 9.997491, 9.868141, 9.775441, 9.710021, 9.663541, 9.628998, 9.600775, 9.574569, 9.547267, 9.516813, 9.4820595, 9.442614, 9.3986845, 9.350919, 9.300255, 9.247799, 9.194727, 9.142099, 9.090922, 9.042042, 8.996116, 8.95362, 8.914868, 8.880016, 8.849077, 8.821942, 8.798403, 8.77819, 8.760988, 8.746467, 8.734293, 8.724142, 8.715709, 8.708714, 8.702908, 8.698078, 8.69404, 8.690638, 8.687743, 8.685253, 8.683086, 8.68118, 8.679486, 8.677966, 8.676596, 8.675353, 8.674224, 8.673197, 8.672268, 8.671427, 8.670671, 8.669994, 8.669394, 

In [14]:
len(predictions[0])

250

In [15]:
len(predictions)

636

In [16]:
type(predictions)

list

In [17]:
from numpy import fft

In [18]:
f = fft.fft(predictions)

In [19]:
f.max()

(19017.402824401855+0j)

In [20]:
from scipy.fftpack import fft

In [21]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack
'''
# Number of samplepoints
N = 600
# sample spacing
T = 1.0 / 800.0
x = np.linspace(0.0, N*T, N)
y = np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
'''
yf = scipy.fftpack.fft(predictions)
xf = np.linspace(0.0, 1.0/(2.0*T), N/2)

fig, ax = plt.subplots()
ax.plot(xf)
plt.show()

NameError: name 'T' is not defined

In [ ]:
len(y)

In [ ]:
import pandas
import matplotlib.pyplot as plt
#import seaborn
%matplotlib inline

# the OP's data
y = predictions
fig, ax = plt.subplots()
ax.plot(y)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack

# Number of samplepoints
N = 600
# sample spacing
T = 1.0 / 800.0
x = np.linspace(0.0, N*T, N)
y = predictions
yf = scipy.fftpack.fft(y)
xf = np.linspace(0.0, 1.0/(2.0*T), N/2)

fig, ax = plt.subplots()
ax.plot(xf, 2.0/N * np.abs(yf[:N//2]))
plt.show()